In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

cwd = os.getcwd() #
fullscrn()

In [2]:
def wndwng0d(x,snstvty=2):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    if snstvty==1 :                        #تشخیص ایپاک پایه
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=200,prominence=300, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=160,prominence=100, width=3)
            #print('class ', i, 'Row = ', j, 'have ',len(peaks),'peaks')
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :  #win=np.append(win, i)# if len(win)==smpl_rte+1 :
                        wndws=np.append(wndws, win)


    if snstvty==2 :                        #تشخیص ایپاک حساس تر
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :
                        wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=17
for i in range (1,18):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,18):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=1000

1 283	2 66	3 20	4 135	5 13	6 21	7 133	8 55	9 13	10 10	11 10	12 10	13 11	14 103	15 62	16 10	17 45	

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])
for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   

print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 2598	2 788	3 224	4 1564	5 158	6 189	7 1371	8 553	9 114	10 104	11 72	12 107	13 156	14 1014	15 633	16 45	17 450	
 max = 2598


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
mx_wndws=int(np.max(cls_wndws)+1)   # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)
rpt_cnn_clsfctn=3                 #تعداد اجرای شبکه عصبی برای میانگین گیری هر حالت 
aug_amnt_Ttl=np.empty((0,7), int) 
rslts=np.array([])
for X in range(3,6):                               #داده افزایی تا چند برابر کلاس اکثریت
    mx_wndws=int(np.max(cls_wndws))   # حداکثر تعداد پنجره ی موجود بین کلاس ها
    mx_wndws=int(X*mx_wndws)          #
    for alpha in range(1,4):
        rprt=np.array([])
        for beta in range(1,4):                              #چند برابر کردن داده افزایی هر کلاس متناسب ضعیف بودن آن کلاس
            if beta==3:
                beta=beta*2

            f_scr=np.array([57,65,74,75,74,62,60,74])     #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            #f_scr=np.array([51,68,73,74,71,56,55,70])      #UnAg,Scl.2,Mag.05,Tm.2,Gs1,frqnc2,GAN,LSTMexpgdo
            f_scr_difrnc=f_scr-f_scr[0]
            for z in range (len(f_scr_difrnc)):
                if f_scr_difrnc[z]<0:
                    f_scr_difrnc[z]=0
            f_scr_difrnc=f_scr_difrnc**beta
            f_scr_rtio=f_scr_difrnc/(sum(f_scr_difrnc))
            aug_amnt=np.empty((0,len(f_scr_rtio)-1), int) 

            scors0=np.array([28,4,71,67,0,75,16,86,56,0,40,81,0,99,99,99,99])         #Class Recalls without Augmentation
            #scors0=np.array([[28,4,71,67,0,75,16,86,56,0,40,81,0,99,100,100,100])     #Class Recalls without Augmentation
            MxMn20fscr=((max(f_scr)-f_scr[0])/20)**0.7
            AgScr=1+alpha*MxMn20fscr*((100-scors0)/(np.max(100-scors0)))**beta        #Class Specific
            
            for i in range (1,cls_num+1):
                aug_amnt=np.append(aug_amnt,[f_scr_rtio[1:]*(mx_wndws*AgScr[i-1]-cls_wndws[i-1])],axis=0)
                #vlum_win=mx_wndws-cls_wndws[i-1]                        #میزان داده افزایی در روش های عمیق (پنجره)
            aug_amnt=np.int16(np.round(aug_amnt))
            aug_amnt_Ttl=np.append(aug_amnt_Ttl,aug_amnt,axis=0)
            #print(np.int16(100*f_scr_rtio))
            #print(aug_amnt)
            rprt=np.empty((0,3),float)
            print(aug_amnt)
            for repeat in range(1,rpt_cnn_clsfctn+1):
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num+1):                            #تعریف آرایه ی پنجره های کلاس ها
                    xtrain=np.append(xtrain,Aug_data(cwd,i,'00',cls_wndws[i-1]),axis=0)
                    if aug_amnt[i-1,0]>0:
                        methd=10
                        sort=-3  #-5=Dis_sam_Cls=qlty1 #-4=var_Dis=qlty2 #-3=DisSm/DisOthr=qlty3  #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5
                        dp_slct_dstrb=4            #1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
                        xtrain=np.append(xtrain,Aug_data(cwd,i,10,aug_amnt[i-1,0]),axis=0)
                    if aug_amnt[i-1,1]>0:
                        methd=20
                        sort=-3
                        dp_slct_dstrb=2
                        xtrain=np.append(xtrain,Aug_data(cwd,i,20,aug_amnt[i-1,1]),axis=0)
                    if aug_amnt[i-1,2]>0:
                        methd=30
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,30,aug_amnt[i-1,2]),axis=0)
                    if aug_amnt[i-1,3]>0:
                        methd=40
                        sort=-1
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,40,aug_amnt[i-1,3]),axis=0)
                    if aug_amnt[i-1,4]>0:
                        methd=50
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,50,aug_amnt[i-1,4]),axis=0)
                    if aug_amnt[i-1,5]>0:
                        methd=60
                        sort=-3
                        dp_slct_dstrb=4                             
                        xtrain=np.append(xtrain,Aug_data(cwd,i,60,aug_amnt[i-1,5]),axis=0) #Scale
                    if aug_amnt[i-1,6]>0:
                        methd=70
                        sort=-3
                        dp_slct_dstrb=4
                        xtrain=np.append(xtrain,Aug_data(cwd,i,70,aug_amnt[i-1,6]),axis=0)
                             
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])

                #for i in range (1,18):             #نرمالسازی داده های افزایشی
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

                wndws_test=np.empty((0,smpl_rte+1), float)
                smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
                i=0
                windws=np.array([])
                for cls in range (1,18):                                 #ساخت پنجره های داده های آموزش اصلی
                    dta=np.array(vars()['ecg'+str(cls)][:int(.1*len(vars()['ecg'+str(cls)])),:]) #انتخاب محدوده ترِین جهت داده افزایی
                    vars()['wndws_tst'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
                    clm0vlu=np.zeros(len(vars()['wndws_tst'+str(cls)]))
                    cls_clm=np.array(clm0vlu)
                    cls_clm=np.int16(cls_clm+cls)
                    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
                    vars()['wndws_tst'+str(cls)]=np.concatenate((vars()['wndws_tst'+str(cls)],cls_clm),axis=1)   

                #print(np.max(wndws_tst))
                for i in range (1,18):                # Normalization test windows
                    cls=i
                    '''print('\n cls', i, ' >> ')
                    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    '''print('after normalizing >>')
                    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))'''

                #print("each class and its windows =")                    #تجمیع کلاس های تست
                cls_wndws=np.array([])
                for cls in range (1,18):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    cls_wndws=np.append(cls_wndws,wns)
                    #print(cls, wns, end='\t')
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)

                #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
                #print('\n max instance in classes of test windows =', mx_wndws)

                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                #VGG16_1D

                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :

                verbose, epochs, batch_size = 0, 1000, btch
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data



                model_crs = Sequential()
                #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
                model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
                model_crs.add(BatchNormalization())
                model_crs.add(MaxPooling1D(pool_size=2,strides=3))

                model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
                model_crs.add(BatchNormalization())
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
                model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
                model_crs.add(MaxPooling1D(pool_size=2,strides=2))

                model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
                #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


                model_crs.add(Flatten())
                model_crs.add(Dropout(0.1))
                model_crs.add(Dense(units=512,activation="relu"))
                #model_crs.add(Dense(units=2048,activation="relu"))
                model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

                #_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

                #print('np.shape(X_test)=',np.shape(X_test))

                #print('Accuracy= ', accuracy)

                #y_pred_vgg = model_crs.predict_classes(X_test)
                predict_x=model_crs.predict(X_test)              # Function 1

                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

                #y_pred_crs = model_crs.predict_classes(X_test)

                cm = confusion_matrix(y_test, y_pred_crs)
                #print(cm)
                acc3=accuracy_score(y_test, y_pred_crs)

                #print(classification_report(y_test, y_pred_crs))

                #sns.heatmap(cm, annot=True)

                #plt.imshow(cm)

                ### Maxcount Predicting Test data
                ecg_test=np.array(ecg1[:int(.1*len(ecg1)),:])
                #print(1,len(ecg_test), end='\t')
                for i in range (2,18):
                    ecg_test=np.concatenate((ecg_test,vars()['ecg'+str(i)][:int(.1*len(vars()['ecg'+str(i)])),:]),axis=0)
                    #print(i,len(vars()['ecg'+str(i)]), end='\t')    
                #print('\n ecg_test shape = ',np.shape(ecg_test))
                i=0
                mlpinputshps=np.array([])
                maxcntlbl=np.array([])
                mlpinput=np.array([])
                mlplbl=np.array([])
                cnnlbl=np.array([])
                peaklens=np.array([])            #array of peaks quantity in each raw signal
                x=np.array([])
                wow=0

                for i in range (len(ecg_test)): #calculating max peaks in row signals
                    peaklens=np.append(peaklens,len(wndwng0d(ecg_test[i])))
                maxpeaks=int(np.max(peaklens))
                #print(maxpeaks)

                prd_lbls_tst=np.empty((0,maxpeaks), int)              #have peaks(windows) lables for each raw signal

                for i in range (len(ecg_test)):
                    windws=np.array([])
                    yyy=np.array([])
                    yy=np.array([])
                    win=np.array([])
                    x=np.array(ecg_test[i,:-1])
                    windws=wndwng0d(x)
                    windws= 2*(windws-mn_aug)/(mx_aug - mn_aug) - 1
                    windws = np.expand_dims(windws, axis=2)
                    yy= model_crs.predict(windws)#yy= Predict each window lable for a raw signal (3600 Pnt) (17 prob in row)
                    yyy=np.int16(maxindx(yy))         # one-hot to class lable
                    maxcntlbl=np.append(maxcntlbl,(np.bincount(yyy).argmax()))
                    zers=np.zeros(int(maxpeaks-len(yyy)))
                    x=np.array(np.append(yyy, zers))
                    prd_lbls_tst=np.append(prd_lbls_tst, [x], axis=0)
                maxcntlbl=np.int16(maxcntlbl)                           
                maxcntlbl=np.int16(maxcntlbl)

                lbls=np.int16(ecg_test[:,-1])

                '''cm = confusion_matrix(lbls, maxcntlbl)
                print(cm)
                acc3=accuracy_score(lbls, maxcntlbl)
                print(classification_report(lbls, maxcntlbl))

                predict_x=model_crs.predict(X_test)# Function 1
                y_pred_crs=maxindx(predict_x)      # function from augment.py to remove 0 index predictions'''

                rprt0=classification_report(lbls, maxcntlbl,output_dict=True)
                rprt_row=np.array([])
                rprt_row=np.append(rprt_row,rprt0['accuracy'])
                rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
                rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
                rprt=np.append(rprt,[rprt_row],axis=0)
            #print('rprt=',rprt)
            maxm=rprt[np.argmax(rprt[:,1])]
            #print('maxm=',maxm)
            avrg=np.mean(rprt,axis=0)
            #print('avrg=',avrg)
            rslts=np.append(rslts,['X '+str(X)+'  alpha '+str(alpha) +'  Beta '+str(beta)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt,axis=0)))],axis=0)
            print('rslts=',rslts[-1])


 max = 2599
[[ 980 2082 2204 2082  612  367 2082]
 [1314 2791 2955 2791  821  493 2791]
 [ 910 1934 2048 1934  569  341 1934]
 [ 811 1724 1825 1724  507  304 1724]
 [1400 2975 3150 2975  875  525 2975]
 [ 886 1883 1994 1883  554  332 1883]
 [1177 2501 2648 2501  736  441 2501]
 [ 777 1651 1748 1651  486  291 1651]
 [1023 2173 2301 2173  639  383 2173]
 [1405 2986 3162 2986  878  527 2986]
 [1136 2413 2555 2413  710  426 2413]
 [ 853 1813 1919 1813  533  320 1813]
 [1400 2976 3151 2976  875  525 2976]
 [ 645 1370 1451 1370  403  242 1370]
 [ 681 1447 1532 1447  425  255 1447]
 [ 736 1564 1656 1564  460  276 1564]
 [ 698 1483 1571 1483  436  262 1483]]
1/1 [==============================] - 0s 13ms/step
rslts= X 3  alpha 1  Beta 1  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[ 558 2519 2825 2519  218   78 2519]
 [ 715 3228 3619 3228  279  101 3228]
 [ 416 1876 2104 1876  162   58 1876]
 [ 419 1890 2119 1890  163   59 1890]
 [ 746 3367 3774 3367  291  105 3367]
 [ 408 1844 206

1/1 [==============================] - 0s 12ms/step
rslts= X 3  alpha 1  Beta 2  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]
[[  21 1921 2707 1921    1    0 1921]
 [  33 3030 4269 3030    2    0 3030]
 [  19 1756 2475 1756    1    0 1756]
 [  19 1731 2439 1731    1    0 1731]
 [  37 3397 4786 3397    2    0 3397]
 [  19 1759 2479 1759    1    0 1759]
 [  25 2302 3244 2302    1    0 2302]
 [  19 1750 2466 1750    1    0 1750]
 [  19 1773 2498 1773    1    0 1773]
 [  37 3399 4789 3399    2    0 3399]
 [  20 1838 2590 1838    1    0 1838]
 [  19 1760 2479 1760    1    0 1760]
 [  37 3399 4790 3399    2    0 3399]
 [  19 1733 2442 1733    1    0 1733]
 [  19 1748 2463 1748    1    0 1748]
 [  19 1762 2483 1762    1    0 1762]
 [  19 1753 2470 1753    1    0 1753]]
1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step
rslts= X 3  alpha 1  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]
[[1686 3582 3793 3582 1053  632 3582]
 [2036 4326 4580 4326 1272  763 4326]
 [1126 2392 2533 2392  703  422 2392]
 [1169 2484 2630 2484  731  438 2484]
 [2095 4451 4713 4451 1309  785 4451]
 [1072 2279 2413 2279  670  402 2279]
 [1863 3959 4192 3959 1165  699 3959]
 [ 919 1953 2068 1953  574  345 1953]
 [1332 2831 2998 2831  833  500 2831]
 [2096 4453 4715 4453 1310  786 4453]
 [1551 3295 3489 3295  969  582 3295]
 [ 991 2106 2230 2106  619  372 2106]
 [2096 4453 4715 4453 1310  786 4453]
 [ 735 1561 1653 1561  459  275 1561]
 [ 741 1574 1667 1574  463  278 1574]
 [ 747 1587 1680 1587  467  280 1587]
 [ 743 1579 1672 1579  464  279 1579]]
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 13ms/step
rslts= X 3  alpha 2  Beta 1  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[ 744 3361 3768 3361  291  105 3361]
 [1046 4724 5296 4724  409  147 4724]
 [ 446 2013 2257 2013  174   63 2013]
 [ 458 2067 2317 2067  179   64 2067]
 [1105 4990 5594 4990  432  155 4990]
 [ 431 1946 2182 1946  168   61 1946]
 [ 886 4002 4487 4002  346  125 4002]
 [ 398 1798 2016 1798  156   56 1798]
 [ 526 2374 2661 2374  205   74 2374]
 [1105 4992 5596 4992  432  155 4992]
 [ 646 2915 3268 2915  252   91 2915]
 [ 412 1861 2086 1861  161   58 1861]
 [1106 4992 5597 4992  432  155 4992]
 [ 380 1718 1926 1718  149   53 1718]
 [ 384 1733 1943 1733  150   54 1733]
 [ 387 1747 1958 1747  151   54 1747]
 [ 385 1738 1948 1738  150   54 1738]]
1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 15ms/step
rslts= X 3  alpha 2  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[  23 2149 3029 2149    1    0 2149]
 [  47 4312 6076 4312    3    0 4312]
 [  19 1757 2476 1757    1    0 1757]
 [  19 1733 2442 1733    1    0 1733]
 [  55 5035 7094 5035    3    0 5035]
 [  19 1759 2479 1759    1    0 1759]
 [  31 2878 4055 2878    2    0 2878]
 [  19 1750 2466 1750    1    0 1750]
 [  19 1785 2515 1785    1    0 1785]
 [  55 5037 7097 5037    3    0 5037]
 [  21 1915 2698 1915    1    0 1915]
 [  19 1760 2479 1760    1    0 1760]
 [  55 5037 7097 5037    3    0 5037]
 [  19 1733 2442 1733    1    0 1733]
 [  19 1748 2463 1748    1    0 1748]
 [  19 1762 2483 1762    1    0 1762]
 [  19 1753 2470 1753    1    0 1753]]
1/1 [==============================] - 0s 14ms/step


...

1/1 [==============================] - 0s 13ms/step
rslts= X 5  alpha 2  Beta 2  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]
[[   4  371  523  371    0    0  371]
 [   9  845 1191  845    1    0  845]
 [   4  343  484  343    0    0  343]
 [   3  317  446  317    0    0  317]
 [  11  998 1406  998    1    0  998]
 [   4  346  488  346    0    0  346]
 [   6  543  766  543    0    0  543]
 [   4  338  476  338    0    0  338]
 [   4  353  498  353    0    0  353]
 [  11 1000 1409 1000    1    0 1000]
 [   4  380  535  380    0    0  380]
 [   4  347  489  347    0    0  347]
 [  11 1000 1409 1000    1    0 1000]
 [   3  320  451  320    0    0  320]
 [   4  335  473  335    0    0  335]
 [   4  349  492  349    0    0  349]
 [   4  341  480  341    0    0  341]]
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step
rslts= X 5  alpha 2  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]
[[ 409  870  921  870  256  154  870]
 [ 530 1126 1192 1126  331  199 1126]
 [ 261  554  586  554  163   98  554]
 [ 266  564  598  564  166  100  564]
 [ 551 1170 1239 1170  344  207 1170]
 [ 246  522  553  522  154   92  522]
 [ 472 1004 1063 1004  295  177 1004]
 [ 197  420  444  420  123   74  420]
 [ 324  688  729  688  202  121  688]
 [ 552 1172 1241 1172  345  207 1172]
 [ 389  827  876  827  243  146  827]
 [ 222  471  499  471  139   83  471]
 [ 552 1172 1241 1172  345  207 1172]
 [ 137  292  309  292   86   52  292]
 [ 144  305  323  305   90   54  305]
 [ 149  318  336  318   93   56  318]
 [ 146  310  328  310   91   55  310]]
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step
rslts= X 5  alpha 3  Beta 1  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[ 173  780  875  780   67   24  780]
 [ 271 1225 1373 1225  106   38 1225]
 [  93  421  472  421   36   13  421]
 [  93  419  469  419   36   13  419]
 [ 291 1312 1471 1312  113   41 1312]
 [  89  404  452  404   35   13  404]
 [ 220  995 1115  995   86   31  995]
 [  78  354  397  354   31   11  354]
 [ 118  533  598  533   46   17  533]
 [ 291 1314 1473 1314  114   41 1314]
 [ 154  695  779  695   60   22  695]
 [  84  379  425  379   33   12  379]
 [ 291 1314 1473 1314  114   41 1314]
 [  70  318  356  318   27   10  318]
 [  74  333  373  333   29   10  333]
 [  77  346  388  346   30   11  346]
 [  75  338  379  338   29   11  338]]
1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step
rslts= X 5  alpha 3  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]
[[   5  417  587  417    0    0  417]
 [  12 1100 1550 1100    1    0 1100]
 [   4  344  484  344    0    0  344]
 [   3  317  447  317    0    0  317]
 [  14 1324 1865 1324    1    0 1324]
 [   4  346  488  346    0    0  346]
 [   7  658  927  658    0    0  658]
 [   4  338  476  338    0    0  338]
 [   4  356  501  356    0    0  356]
 [  14 1326 1868 1326    1    0 1326]
 [   4  395  557  395    0    0  395]
 [   4  347  489  347    0    0  347]
 [  14 1326 1868 1326    1    0 1326]
 [   3  320  451  320    0    0  320]
 [   4  335  473  335    0    0  335]
 [   4  349  492  349    0    0  349]
 [   4  341  480  341    0    0  341]]
35/35 [==============================] - 0s 3ms/step


1/1 [==============================] - 0s 13ms/step
rslts= X 5  alpha 3  Beta 6  average= [98 98 98]  max= [98 99 98]  var= [0 0 0]


In [6]:
rslts

array(['X 3  alpha 1  Beta 1  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 3  alpha 1  Beta 2  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 3  alpha 1  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 3  alpha 2  Beta 1  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 3  alpha 2  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 3  alpha 2  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 3  alpha 3  Beta 1  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 3  alpha 3  Beta 2  average= [99 99 99]  max= [100 100 100]  var= [0 0 0]',
       'X 3  alpha 3  Beta 6  average= [98 99 98]  max= [98 99 98]  var= [0 0 0]',
       'X 4  alpha 1  Beta 1  average= [97 99 97]  max= [98 99 98]  var= [0 0 0]',
       'X 4  alpha 1  Beta 2  average= [97 99 97]  max= [98 99 98]  var= [0 0 0]',
       'X 4  alpha 1  Beta 6  average= [98 98 98]  max= [98 99 98]  var= [0

In [7]:
cls_wndws[i-1]

IndexError: index 94 is out of bounds for axis 0 with size 17

In [ ]:
np.shape(f_scr_rtio)

In [ ]:
AgScr[i-1]

In [ ]:
os._exit(00)
srt2